In [24]:
from bs4 import BeautifulSoup
import os
import pandas as pd
!pip install fuzzywuzzy[speedup]
from fuzzywuzzy import process
import re

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
directory = '/content/drive/MyDrive/PEK/ОТР'

In [27]:
paragraphs_dataframes = []

In [28]:
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)

    # Check if the file exists and is an HTML file
    if os.path.isfile(file_path) and filename.endswith('.html'):
        # Load the HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Remove crossed-out text elements
        for tag in soup.find_all(['s', 'strike', 'del']):
            tag.decompose()

        # Extract all paragraphs and concatenate their text
        paragraphs = soup.find_all('p')
        tables = soup.find_all('td')
        paragraphs_text = ' '.join(p.get_text(strip=True) for p in paragraphs)
        table_items_text = ' '.join(table.get_text(strip=True) for table in tables)
        combined_text = paragraphs_text + ' ' + table_items_text

        # Create a DataFrame for the concatenated paragraphs
        df_paragraphs = pd.DataFrame([[os.path.splitext(filename)[0], combined_text]],
                                     columns=['Query', 'Answer'])

        # Append the DataFrame to the list
        paragraphs_dataframes.append(df_paragraphs)

In [29]:
combined_paragraphs_df = []
combined_paragraphs_df = pd.concat(paragraphs_dataframes, ignore_index=True)

In [30]:
display(combined_paragraphs_df)

,Query,Answer
0,ОТР (DPEGAS-22208),Описание технической реализации DPEGAS-22208-П...
1,ОТР (DPEGAS-22199),Описание технической реализации DPEGAS-22199-П...
2,ОТР (DPEGAS-22271),Описание технической реализации DPEGAS-22271-П...
3,ОТР (DPEGAS-22259),Описание технической реализации DPEGAS-22259-П...
4,ОТР (DPEGAS-22332),Описание технической реализации DPEGAS-22332-...
...,...,...
210,ОТР (DPEGAS-36120),Описание технической реализации DPEGAS-36120...
211,ОТР (DPEGAS-36085),Описание технической реализации DPEGAS-36085-П...
212,ОТР (DPEGAS-36210),Описание технической реализации DPEGAS-36210-П...
213,ОТР (DPEGAS-36187),Описание технической реализации DPEGAS-36187-П...


In [31]:
patterns_to_remove = ['… СТАТУС', 'ОЦЕНОЧНАЯ ЛИЦЕНЗИЯВам нравится Confluence? Пожалуйста, рассмотритедля приобретения егосегодня.']
for pattern in patterns_to_remove:
    combined_paragraphs_df['Answer'] = combined_paragraphs_df['Answer'].str.replace(pattern, '', regex=False)

display(combined_paragraphs_df)

,Query,Answer
0,ОТР (DPEGAS-22208),Описание технической реализации DPEGAS-22208-П...
1,ОТР (DPEGAS-22199),Описание технической реализации DPEGAS-22199-П...
2,ОТР (DPEGAS-22271),Описание технической реализации DPEGAS-22271-П...
3,ОТР (DPEGAS-22259),Описание технической реализации DPEGAS-22259-П...
4,ОТР (DPEGAS-22332),Описание технической реализации DPEGAS-22332-...
...,...,...
210,ОТР (DPEGAS-36120),Описание технической реализации DPEGAS-36120...
211,ОТР (DPEGAS-36085),Описание технической реализации DPEGAS-36085-П...
212,ОТР (DPEGAS-36210),Описание технической реализации DPEGAS-36210-П...
213,ОТР (DPEGAS-36187),Описание технической реализации DPEGAS-36187-П...


In [32]:
# Function to remove the artifact at the end
def remove_artifacts(text):
    return re.sub(r'\s*\{[^{}]*\}$', '', text).strip()

# Apply the function to the 'Answer' column
combined_paragraphs_df['Answer'] = combined_paragraphs_df['Answer'].apply(remove_artifacts)

In [33]:
display(combined_paragraphs_df)

,Query,Answer
0,ОТР (DPEGAS-22208),Описание технической реализации DPEGAS-22208-П...
1,ОТР (DPEGAS-22199),Описание технической реализации DPEGAS-22199-П...
2,ОТР (DPEGAS-22271),Описание технической реализации DPEGAS-22271-П...
3,ОТР (DPEGAS-22259),Описание технической реализации DPEGAS-22259-П...
4,ОТР (DPEGAS-22332),Описание технической реализации DPEGAS-22332-П...
...,...,...
210,ОТР (DPEGAS-36120),Описание технической реализации DPEGAS-36120-...
211,ОТР (DPEGAS-36085),Описание технической реализации DPEGAS-36085-П...
212,ОТР (DPEGAS-36210),Описание технической реализации DPEGAS-36210-П...
213,ОТР (DPEGAS-36187),Описание технической реализации DPEGAS-36187-П...


In [34]:
links=pd.read_csv('/content/drive/MyDrive/PEK/OTR.txt', sep='/t', header=None)
display(links) #links

<ipython-input-34-3ca4bfaf58d8>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  links=pd.read_csv('/content/drive/MyDrive/PEK/OTR.txt', sep='/t', header=None)


,0
0,name;value
1,ОТР (DARCH-1059);http://confluence/pages/viewp...
2,ОТР (DBUH-1106);http://confluence/pages/viewpa...
3,ОТР (DBUH-1109);http://confluence/pages/viewpa...
4,ОТР (DBUH-1117);http://confluence/pages/viewpa...
...,...
211,ОТР (RND-910);http://confluence/pages/viewpage...
212,ОТР (RND-933);http://confluence/pages/viewpage...
213,ОТР (RND-937);http://confluence/pages/viewpage...
214,ОТР (RND-939);http://confluence/pages/viewpage...


In [35]:
# Split the unnamed column by the last ";"

# Function to split by the last semicolon
def split_by_last_semicolon(text):
    parts = text.rsplit(';', 1)
    return parts if len(parts) == 2 else [text, '']

# Apply the function to split the unnamed column
links_split = links.iloc[:, 0].apply(split_by_last_semicolon).apply(pd.Series)

# Rename the columns
links_split.columns = ['File', 'Link']

display(links_split)

,File,Link
0,name,value
1,ОТР (DARCH-1059),http://confluence/pages/viewpage.action?pageId...
2,ОТР (DBUH-1106),http://confluence/pages/viewpage.action?pageId...
3,ОТР (DBUH-1109),http://confluence/pages/viewpage.action?pageId...
4,ОТР (DBUH-1117),http://confluence/pages/viewpage.action?pageId...
...,...,...
211,ОТР (RND-910),http://confluence/pages/viewpage.action?pageId...
212,ОТР (RND-933),http://confluence/pages/viewpage.action?pageId...
213,ОТР (RND-937),http://confluence/pages/viewpage.action?pageId...
214,ОТР (RND-939),http://confluence/pages/viewpage.action?pageId...


In [36]:
# Function to extract numeric part
def extract_number(text):
    match = re.search(r'\d+', text)
    return match.group(0) if match else None

In [37]:
# Extract numbers from Query and File columns
combined_paragraphs_df['Key'] = combined_paragraphs_df['Query'].apply(extract_number)
links_split['Key'] = links_split['File'].apply(extract_number)

In [40]:
# Merge DataFrames on the extracted numeric key
merged_otr_df = pd.merge(combined_paragraphs_df, links_split, on='Key', how='left')

In [41]:
# Drop the Key column as it's no longer needed
merged_otr_df.drop(columns=['Key'], inplace=True)

In [44]:
merged_otr_df = merged_otr_df.drop(columns=['File'])

In [45]:
merged_otr_df['Department'] = 'OTR'

In [46]:
merged_otr_df.to_csv('/content/drive/MyDrive/merged_otr_df.csv', index=False)